In [1]:
!pip install statsmodels==0.12.1
!pip install fbprophet
!pip install statsmodels

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
!pip install statsmodels==0.12.2

     |████████████████████████████████| 9.5 MB 5.1 MB/s eta 0:00:01
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.1
    Uninstalling statsmodels-0.12.1:
      Successfully uninstalled statsmodels-0.12.1
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')
import gc
import math
import os.path
import time
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
from tqdm.auto import tqdm
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error
import copy
from fbprophet import Prophet
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error as mse
%matplotlib inline

In [4]:
train_x = pd.read_csv('../input/bitcoin3/train_x_df.csv')
test_x = pd.read_csv('../input/bitcoin3/test_x_df.csv')
train_y = pd.read_csv('../input/bitcoin3/train_y_df.csv')
submission = pd.read_csv('../input/bitcoin3/sample_submission.csv')

In [5]:
train_x

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785
...,...,...,...,...,...,...,...,...,...,...,...,...
10942015,7928,1375,8,0.999708,1.000073,0.999562,0.999854,1.048120,143.449982,0.401812,0.612980,83.898582
10942016,7928,1376,8,0.999708,1.000000,0.999489,0.999781,0.706883,96.735909,0.409117,0.501885,68.683640
10942017,7928,1377,8,0.999708,1.000000,0.999489,0.999781,1.643578,224.917908,0.620982,0.788707,107.937714
10942018,7928,1378,8,0.999708,1.000146,0.999489,1.000000,1.204417,164.840317,0.752484,0.869763,119.042564


In [6]:
# VWAP
def vwap(df):
    df['volume_price'] = (df['open']) * (df['volume'])
    df['volume_price_sum'] = df.groupby(['sample_id'])['volume_price'].apply(lambda x: x.cumsum())
    df['volume_sum'] = df.groupby(['sample_id'])['volume'].apply(lambda x: x.cumsum())
    df['vwap'] = df['volume_price_sum'] / df['volume_sum'] # open이 vwap보다 크면 상승세, vwap보다 작으면 하락세
    df["diff"] = df["open"] - df["vwap"] # diff가 양수이면 상승세, 음수이면 하락세
    return df['diff']

# RSI
def RSI(df):
    df['up_width'] = np.where(df.groupby(['sample_id'])['open'].diff() > 0, np.abs(df.groupby(['sample_id'])['open'].diff()), 0)
    df['down_width'] = np.where(df.groupby(['sample_id'])['open'].diff() < 0, df.groupby(['sample_id'])['open'].diff() * (-1), 0)
    ud_df = pd.DataFrame()
    ud_df["sample_id"] = df["sample_id"]
    ud_df["up_width"] = df["up_width"]
    ud_df["down_width"] = df["down_width"]
    df['AU'] = ud_df.groupby(['sample_id'])['up_width'].rolling(window=14,min_periods=6).mean().reset_index()["up_width"] #6 or 14
    df['AD'] = ud_df.groupby(['sample_id'])['down_width'].rolling(window=14,min_periods=6).mean().reset_index()["down_width"] #6 or 14
    df['RSI'] = df["AU"] / (df["AU"] + df["AD"]) * 100 # 70이상을 초과매수 국면으로(매도 포지션) , 30이하를 초과매도 국면으로(매수 포지션)
    # 스토캐스틱 
    df['Stoch_RSI'] = (df['RSI'] - df['RSI'].min())/(df['RSI'].max() - df['RSI'].min()) # 20이하 과매도(악세 또는 하락추세), 80이상 과매수(강세 또는 상향 추세)
    return df['RSI'],df['Stoch_RSI']


# 단순 이동평균(SMA)
# 단기이평선(5,20,60) , 중장기이평선(60,120) , 장기이평선(그 이상)
# (단기이동평균선(5일,20일) > 장기이동평균선(60일,120일) : 콜든크로스(상향 돌파 시,매수), 단기이동평균선(5일,20일) < 장기이동평균선(60일,120일) : 데드크로스(하향 돌파 시,매도))
def sma(df):
    df['sma5'] = df['close'].rolling(window = 5).mean()
    df['sma20'] = df['close'].rolling(window = 60).mean()
    return df
    
# 지수 이동평균(EMA)
# 단순 이동평균과의 차이점 : 최근 데이터(오늘의 종가)에 높은 가중치를 주고
def ema12(df,period=12):
    df['ema12'] = df['close'].ewm(span = period,adjust=False).mean()
    return df

def ema26(df,period=26):
    df['ema26'] = df['close'].ewm(span = period,adjust=False).mean()
    return df
   
"""
# MACD
# 필수 보조지표
def macd(df,period_long = 26, period_short=12, period_signal=9):
    shortema = ema(df,period_short)
    longema = ema(df,period_long)
    df['macd'] = shortema - longema
    df['signal'] = ema(df,period_signal)
    df['macd_oscillator'] = df['macd']- df['signal']
    return df
"""


def macd(df): # short = 단기 이동평균선(12일), long = 장기 이동평균선(26일) signal = MACD값을 지수이동평균한 값 = 기본적으로 9일로 설정
    df['MACD'] = df['close'].ewm(span = 26,adjust=False).mean() - df['close'].ewm(span = 12,adjust=False).mean()
    df['MACD_Signal'] = df['close'].ewm(span = 9,adjust=False).mean()
    df['MACD_Oscillator'] = df['MACD'] - df['MACD_Signal'] # 오실리에이터 = MACD - Signal
    # 오실리에이터가 0보다 크면 상향돌파(매수신호), 0보다 작으면 하향돌파(매도신호)
    return df

#퓨엘지수
def puel(df):
    df['puel'] = df['open']/df['open'].rolling(window=365).mean()
    return df
    
# 변동성 + 상승장 투자전략
def coin_rule(df):    
    df['ma5'] = df['close'].rolling(window=5).mean().shift(1)
    df['range'] = (df['high'] - df['low']) * 0.5
    df['target'] = df['open'] + df['range'].shift(1)
    df['bull'] = df['open'] > df['ma5']
    return df

# 볼린저 밴드 (대형주들이 많이 사용)
# 나의 생각 -> 하한선과 중심선 사이 or 중심선과 상한선 사이를 지나고 있으면 매수 타이밍
def bol(df):    
    w= 20 # 기준 이동평균일 
    k= 2 # 기준 상수
    
    #중심선 (MBB) : n일 이동평균선
    df["bol ma"]=df["close"].rolling(w).mean() # 중심선을 지나면 무조건 매수 시기
    df["bol std"]=df["close"].rolling(w).std()
 
    #상한선 (UBB) : 중심선 + (표준편차 × K)
    #하한선 (LBB) : 중심선 - (표준편차 × K)
    df['bol upper'] = df['bol ma'] + (df['bol std'] * k) # 상한선(지지선)을 이탈하면 매수 금지
    df['bol lower'] = df['bol ma'] - (df['bol std'] * k) # 하한선(지지선)을 이탈하면 매수 금지
    return df

In [7]:
# train_x, train_y 를 sample_id 기준 하나로 합쳐서, train_z를 만들어줍니다.
# 사실 train x, y를 합치지 않아도 괜찮습니다만, 연속되는 변수 vwap, rsi, 등등을 만들기 위해 x, y를 합치고, 변수를 생성하고, x, y를 다시 분리하는 작업을 진행했습니다.
train_x["is_x"] = 1
train_y["is_x"] = 0
train_x_y = [train_x, train_y]
train_list = [x.set_index('sample_id') for x in train_x_y]

train_z = pd.concat(train_list, axis=0).rename_axis('sample_id').reset_index()

In [8]:
# 데이터 프레임에서 sample_id 에 따른 지표를 반환하는 함수입니다.

def get_Indicators(df,sample_id,col):
    return df[df['sample_id'] == sample_id][col].values

**MODELING**

In [9]:
# train_z
vwap(train_z)
RSI(train_z)
macd(train_z)
sma(train_z)
ema12(train_z)
ema26(train_z)
puel(train_z)
coin_rule(train_z)
bol(train_z)


# test_x
vwap(test_x)
RSI(test_x)
macd(test_x)
sma(test_x)
ema12(test_x)
ema26(test_x)
puel(test_x)
coin_rule(test_x)
bol(test_x)

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,...,ema26,puel,ma5,range,target,bull,bol ma,bol std,bol upper,bol lower
0,7929,0,1,0.941549,0.941773,0.940431,0.941176,0.169500,85.595390,0.089465,...,0.941176,NaN,NaN,0.000671,NaN,False,NaN,NaN,NaN,NaN
1,7929,1,1,0.941586,0.941736,0.940282,0.940897,0.237560,119.955322,0.119287,...,0.941156,NaN,NaN,0.000727,0.942257,False,NaN,NaN,NaN,NaN
2,7929,2,1,0.941270,0.941586,0.938940,0.940934,0.231588,116.848953,0.149109,...,0.941139,NaN,NaN,0.001323,0.941997,False,NaN,NaN,NaN,NaN
3,7929,3,1,0.940971,0.941363,0.939052,0.940319,0.238199,120.162766,0.201297,...,0.941079,NaN,NaN,0.001156,0.942295,False,NaN,NaN,NaN,NaN
4,7929,4,1,0.940077,0.940561,0.939760,0.939909,0.082302,41.511501,0.055916,...,0.940992,NaN,NaN,0.000401,0.941232,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048795,8688,1375,5,1.005905,1.007405,1.005392,1.006122,32184.292969,23957.617188,66.215317,...,1.003846,0.993160,1.004646,0.001007,1.007398,True,1.005486,0.002816,1.011118,0.999854
1048796,8688,1376,5,1.006243,1.006554,1.004040,1.004608,46183.425781,34342.394531,93.241982,...,1.003903,0.993584,1.005084,0.001257,1.007250,True,1.005386,0.002810,1.011006,0.999767
1048797,8688,1377,5,1.004608,1.005284,1.002865,1.002865,55310.468750,41101.046875,97.295982,...,1.003826,0.992071,1.005065,0.001209,1.005865,False,1.005103,0.002762,1.010626,0.999580
1048798,8688,1378,5,1.002865,1.002905,1.000013,1.000054,114685.742188,85007.203125,127.025307,...,1.003546,0.990460,1.004994,0.001446,1.004074,False,1.004603,0.002723,1.010049,0.999157


In [10]:
train_z.isnull().sum()

sample_id                0
time                     0
coin_index               0
open                     0
high                     0
low                      0
close                    0
volume                   0
quote_av                 0
trades                   0
tb_base_av               0
tb_quote_av              0
is_x                     0
volume_price             0
volume_price_sum         0
volume_sum               0
vwap                  2125
diff                  2125
up_width                 0
down_width               0
AU                   39645
AD                   39645
RSI                 108188
Stoch_RSI           108188
MACD                     0
MACD_Signal              0
MACD_Oscillator          0
sma5                     4
sma20                   59
ema12                    0
ema26                    0
puel                   364
ma5                      5
range                    0
target                   1
bull                     0
bol ma                  19
b

In [11]:
test_x.isnull().sum()

sample_id              0
time                   0
coin_index             0
open                   0
high                   0
low                    0
close                  0
volume                 0
quote_av               0
trades                 0
tb_base_av             0
tb_quote_av            0
volume_price           0
volume_price_sum       0
volume_sum             0
vwap                   0
diff                   0
up_width               0
down_width             0
AU                  3800
AD                  3800
RSI                 3800
Stoch_RSI           3800
MACD                   0
MACD_Signal            0
MACD_Oscillator        0
sma5                   4
sma20                 59
ema12                  0
ema26                  0
puel                 364
ma5                    5
range                  0
target                 1
bull                   0
bol ma                19
bol std               19
bol upper             19
bol lower             19
dtype: int64

In [12]:
# 4. train x와 y를 분리합니다.
train_x = train_z[train_z["is_x"] == 1]
train_y = train_z[train_z["is_x"] == 0]

# 결측치 처리
train_z.fillna(method='ffill',inplace=True)
train_z.fillna(method='bfill',inplace=True)
test_x.fillna(method='ffill',inplace=True)
test_x.fillna(method='bfill',inplace=True)

split_drop_cols = ["is_x"]

train_x = train_x.drop(columns=split_drop_cols, axis=1)
train_y = train_y.drop(columns=split_drop_cols, axis=1)

In [13]:
train_x

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,...,ema26,puel,ma5,range,target,bull,bol ma,bol std,bol upper,bol lower
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,...,0.992966,NaN,NaN,0.000344,NaN,False,NaN,NaN,NaN,NaN
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,...,0.992947,NaN,NaN,0.000417,0.993600,False,NaN,NaN,NaN,NaN
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,...,0.993086,NaN,NaN,0.001178,0.993165,False,NaN,NaN,NaN,NaN
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,...,0.993227,NaN,NaN,0.000598,0.995957,False,NaN,NaN,NaN,NaN
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,...,0.993342,NaN,NaN,0.000526,0.995160,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10942015,7928,1375,8,0.999708,1.000073,0.999562,0.999854,1.048120,143.449982,0.401812,...,0.998973,1.005597,0.999620,0.000256,0.999890,True,0.998875,0.000866,1.000606,0.997144
10942016,7928,1376,8,0.999708,1.000000,0.999489,0.999781,0.706883,96.735909,0.409117,...,0.999033,1.005575,0.999693,0.000256,0.999963,True,0.998995,0.000811,1.000618,0.997373
10942017,7928,1377,8,0.999708,1.000000,0.999489,0.999781,1.643578,224.917908,0.620982,...,0.999089,1.005553,0.999722,0.000256,0.999963,False,0.999138,0.000667,1.000472,0.997804
10942018,7928,1378,8,0.999708,1.000146,0.999489,1.000000,1.204417,164.840317,0.752484,...,0.999156,1.005533,0.999810,0.000329,0.999963,False,0.999273,0.000535,1.000343,0.998203


In [14]:
train_y

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,...,ema26,puel,ma5,range,target,bull,bol ma,bol std,bol upper,bol lower
10942020,0,0,0,1.000326,1.002683,0.999420,1.001160,6848.100586,18914.638672,56.925312,...,0.999368,1.006117,0.999898,0.001632,1.000545,True,0.999484,0.000628,1.000740,0.998229
10942021,0,1,0,1.001342,1.001450,0.999347,1.000000,5056.033691,13946.318359,42.059464,...,0.999414,1.007118,1.000159,0.001052,1.002973,True,0.999576,0.000556,1.000688,0.998464
10942022,0,2,0,1.000218,1.001015,0.999637,1.000689,2529.303955,6977.748047,29.006527,...,0.999509,1.005970,1.000203,0.000689,1.001269,True,0.999647,0.000603,1.000853,0.998440
10942023,0,3,0,1.000689,1.001632,1.000254,1.001305,2488.415527,6867.863281,20.667150,...,0.999642,1.006428,1.000384,0.000689,1.001378,True,0.999752,0.000697,1.001147,0.998357
10942024,0,4,0,1.001595,1.001849,1.000906,1.001052,446.595367,1233.619995,6.889050,...,0.999746,1.007321,1.000646,0.000471,1.002284,True,0.999849,0.000738,1.001325,0.998373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11893495,7928,115,8,0.995251,0.995470,0.994813,0.995397,0.477114,64.997147,0.394506,...,0.995955,0.996176,0.995485,0.000329,0.995507,False,0.995423,0.000602,0.996627,0.994219
11893496,7928,116,8,0.995105,0.995397,0.994667,0.995397,1.152308,156.951431,0.533314,...,0.995913,0.996071,0.995427,0.000365,0.995434,False,0.995365,0.000539,0.996442,0.994287
11893497,7928,117,8,0.995178,0.996420,0.994959,0.996128,1.609954,219.404938,0.759790,...,0.995929,0.996186,0.995354,0.000731,0.995544,False,0.995372,0.000548,0.996469,0.994275
11893498,7928,118,8,0.996347,0.996347,0.995324,0.995617,0.777472,105.957024,0.379895,...,0.995906,0.997395,0.995470,0.000511,0.997078,True,0.995368,0.000546,0.996461,0.994275


In [15]:
test_x['time'].unique()

array([   0,    1,    2, ..., 1377, 1378, 1379])

In [16]:
#c= train_x['high'][(train_x['bol upper'] > train_x['high']) & (train_x['high'] >= train_x['bol ma'])] # 매수 상관 x
#b= train_x['high'][(train_x['bol upper'] < train_x['high']) & (train_x['high'] >= train_x['bol ma'])] # 매수 금지 = 0
#c = train_x['high'][train_x['high'] == train_x['bol ma']] # 매수 타이밍
#print(a)
#print(b)
#print(c)

In [17]:
#a = train_x['low'][(train_x['bol lower'] < train_x['low']) & (train_x['low'] =< train_x['bol ma'])] # 매수 상관 x
#b = train_x['low'][(train_x['bol lower'] > train_x['low']) & (train_x['low'] =< train_x['bol ma'])] # 매수 금지 = 0
#c = train_x['low'][train_x['low'] == train_x['bol ma']] # 매수 타이밍
#print(a)
#print(b)
#print(c)

In [18]:
import statsmodels.api as sm

result = []

for sample_id in tqdm(test_x['sample_id'].unique()):
    high_series = get_Indicators(test_x,sample_id,'high')
    low_series = get_Indicators(test_x,sample_id,'low')
    open_series = get_Indicators(test_x,sample_id,'open')
    diff_x = get_Indicators(test_x,sample_id,'diff')
    vwap_series = get_Indicators(test_x,sample_id,'vwap')
    rsi_series = get_Indicators(test_x,sample_id,'RSI')
    stoch_rsi_series = get_Indicators(test_x,sample_id,'Stoch_RSI')
    sma5_series = get_Indicators(test_x,sample_id,'sma5')
    sma20_series = get_Indicators(test_x,sample_id,'sma20')
    ema12_series = get_Indicators(test_x,sample_id,'ema12')
    ema26_series = get_Indicators(test_x,sample_id,'ema26')
    macd_series = get_Indicators(test_x,sample_id,'MACD_Oscillator')
    puel_series = get_Indicators(test_x,sample_id,'puel')
    bull_series = get_Indicators(test_x,sample_id,'bull')
    bol_ma_series = get_Indicators(test_x,sample_id,'bol ma')
    bol_upper_series = get_Indicators(test_x,sample_id,'bol upper')
    bol_lower_series = get_Indicators(test_x,sample_id,'bol lower')
    
    try:
        try:
            print('order=(1,0,0)')
            model = ARIMA(diff_x,order=(1,0,0)) #1,0,0
            res = model.fit()
        except:
            print("order=(4,0,1)")
            model = ARIMA(diff_x,order=(4,0,1)) # 4 0 1
            res = model.fit(trend='nc',full_output=True, disp=True)
    except:
        model = ARIMA(diff_x,order=(3,0,1)) # 3 0 1
        res = model.fit(trend='nc',full_output=True, disp=True)
        print("샘플은 수렴하지 않습니다. order=(3,0,1)")
        #print("샘플은 수렴하지 않습니다.")
        #pass
        
    model_forecast = res.predict(1,120,typ='levels')
    sell_time = np.argmax(model_forecast)
    max_val = np.max(model_forecast)
    vwap_last_val = vwap_series[1379]
    rsi_last_val = rsi_series[1379]
    stoch_rsi_last_val = stoch_rsi_series[1379]
    macd_last_val = macd_series[1379]
    sma5_last_val = sma5_series[1379]
    sma20_last_val = sma20_series[1379]
    ema12_last_val = ema12_series[1379]
    ema26_last_val = ema26_series[1379]
    puel_last_val = puel_series[1379]
    bull_last_val = bull_series[1379]
    bol_upper_series = bol_upper_series[1379]
    bol_lower_series = bol_lower_series[1379]
    bol_ma_series = bol_ma_series[1379]
    high_series = high_series[1379]
    low_series = low_series[1379]
    open_series = open_series[1379]
    
    # 4. 투자 전략
    buy_quantity = 0

    # 1) 최대값이 0 보다 크면 가격이 vwap 보다 크다는 의미로, 투자합니다.
    if  max_val > 0:
        buy_quantity = 1
        
    # =============
    # 매수 금지 전략
    # =============
    
    # 2) 만약 vwap 마지막 값이, 1보다 크면 가격이 1보다 작다는 의미로 하향세이기 때문에 투자하지 않습니다.
    if vwap_last_val > 1 and sell_time <= 60:
        buy_quantity = 0

    # 3) 만약 rsi의 값이 65/70 보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
    if rsi_last_val > 65 and sell_time <= 60:
        buy_quantity = 0
        
    # 4) 만약 stoch_rsi의 값이 75/80보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
    if stoch_rsi_last_val > 75 and sell_time <= 60:
        buy_quantity = 0
    
    if sma5_last_val < sma20_last_val and sell_time <= 60:
        buy_quantity = 0
        
    if ema12_last_val < ema26_last_val and sell_time <= 60:
        buy_quantity = 0
    
    if macd_last_val > 0 and sell_time <= 60:
        buy_quantity = 0
    
    if puel_last_val > 1 and sell_time <= 60:
        buy_quantity = 0
        
    #if bull_last_val == False and sell_time < 56:
     #   buy_quantity = 0
    
    if bol_upper_series < high_series and sell_time <= 60:
        buy_quantity = 0
    
    if bol_lower_series > low_series and sell_time <= 60:
        buy_quantity = 0
        
    # =============
    # 매수 진행 전략
    # =============
        
    if  bol_ma_series == high_series:
        buy_quantity = 1
    
    if bol_ma_series == low_series:
        buy_quantity = 1
    
    if bol_ma_series == open_series:
        buy_quantity = 1
    
# all < 50 / < 56
        
    # 5. 결과
    result_list = [
                   sample_id,
                   buy_quantity,
                   sell_time
                  ]

    result.append(result_list)

    
# 1. 학습 결과를 데이터 프레임으로 만듭니다.

submit_columns = [
                  "sample_id", 
                  "buy_quantity", 
                  "sell_time"
                  ]


submission = pd.DataFrame(data=result, columns=submit_columns)

  0%|          | 0/760 [00:00<?, ?it/s]

order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=

/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=(1,0,0)
order=

In [19]:
"""       
    for i in range(0,1380):
        model_forecast = res.predict(1,120,typ='levels')
        sell_time = np.argmax(model_forecast)
        max_val = np.max(model_forecast)
        vwap_last_val = vwap_series[i]
        rsi_last_val = rsi_series[i]
        stoch_rsi_last_val = stoch_rsi_series[i]
        macd_last_val = macd_series[i]
        sma5_last_val = sma5_series[i]
        sma20_last_val = sma20_series[i]
        ema12_last_val = ema12_series[i]
        ema26_last_val = ema26_series[i]
        #puel_last_val = puel_series[i]
        bull_last_val = bull_series[i]
        #bol_upper_series = bol_upper_series[i]
        #bol_lower_series = bol_lower_series[i]
        #high_series = high_series[i]
        #low_series = low_series[i]
        
        buy_quantity = 0
        
        if  max_val > 0:
            buy_quantity = 1
        
        # 2) 만약 vwap 마지막 값이, 1보다 크면 가격이 1보다 작다는 의미로 하향세이기 때문에 투자하지 않습니다.
        if vwap_last_val > 1 and sell_time < 56:
            buy_quantity = 0
            
        # 3) 만약 rsi의 값이 65/70 보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
        if rsi_last_val > 65 and sell_time < 56:
            buy_quantity = 0
        
        # 4) 만약 stoch_rsi의 값이 75/80보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
        if stoch_rsi_last_val > 75 and sell_time < 56:
            buy_quantity = 0
    
        if sma5_last_val < sma20_last_val and sell_time < 56:
            buy_quantity = 0
        
        if ema12_last_val < ema26_last_val and sell_time < 56:
            buy_quantity = 0
    
        if macd_last_val > 0 and sell_time < 56:
            buy_quantity = 0
    
        #if puel_last_val > 1 and sell_time < 56:
        #    buy_quantity = 0
        
        if bull_last_val == False and sell_time < 56:
            buy_quantity = 0
        
        #if bol_upper_series <= high_series and sell_time < 56:
        #    buy_quantity = 1
    
        #if bol_lower_series >= low_series and sell_time < 50:
        #    buy_quantity = 1
            
        # 5. 결과
        result_list = [sample_id,buy_quantity,sell_time]

        result.append(result_list)
        """


#sub = submission.sort_values(by=['sample_id','sell_time'],ascending=True)
#sub
#sub = sub.drop_duplicates(['sample_id'], keep = 'first')
#sub.to_csv('codethanks_day2_fix1.csv',index=False)

#sub['buy_quantity'].value_counts()

"       \n    for i in range(0,1380):\n        model_forecast = res.predict(1,120,typ='levels')\n        sell_time = np.argmax(model_forecast)\n        max_val = np.max(model_forecast)\n        vwap_last_val = vwap_series[i]\n        rsi_last_val = rsi_series[i]\n        stoch_rsi_last_val = stoch_rsi_series[i]\n        macd_last_val = macd_series[i]\n        sma5_last_val = sma5_series[i]\n        sma20_last_val = sma20_series[i]\n        ema12_last_val = ema12_series[i]\n        ema26_last_val = ema26_series[i]\n        #puel_last_val = puel_series[i]\n        bull_last_val = bull_series[i]\n        #bol_upper_series = bol_upper_series[i]\n        #bol_lower_series = bol_lower_series[i]\n        #high_series = high_series[i]\n        #low_series = low_series[i]\n        \n        buy_quantity = 0\n        \n        if  max_val > 0:\n            buy_quantity = 1\n        \n        # 2) 만약 vwap 마지막 값이, 1보다 크면 가격이 1보다 작다는 의미로 하향세이기 때문에 투자하지 않습니다.\n        if vwap_last_val > 1 and sell_t

In [20]:
submission['buy_quantity'].value_counts()

1    419
0    341
Name: buy_quantity, dtype: int64

In [21]:
submission.to_csv('codethanks_day_last_2.csv',index=False)